In [2]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

print("Torch version:", torch.__version__)

# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)


# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Torch version: 1.2.0
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[{'boxes': tensor([[216., 188., 381., 358.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([1], device='cuda:0'), 'area': tensor([28050.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[140., 188., 306., 293.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([0], device='cuda:0'), 'area': tensor([17430.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 59., 113., 325., 542.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([3], device='cuda:0'), 'area': tensor([114114.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[144.,  78., 487., 342.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([5], device='cuda:0'), 'area': tensor([90552.], device='cuda:0'), 'iscrowd': te

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/user/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:37<00:00, 2.70MB/s]


Epoch: 0/10
Iteration: 1/6, Loss: 1.4161217212677002
Iteration: 2/6, Loss: 1.391079068183899
Iteration: 3/6, Loss: 1.3717596530914307
Iteration: 4/6, Loss: 1.2866002321243286
Iteration: 5/6, Loss: 1.24117112159729
Iteration: 6/6, Loss: 1.1091028451919556
Epoch: 1/10
Iteration: 1/6, Loss: 0.9804332852363586
Iteration: 2/6, Loss: 1.0027178525924683
Iteration: 3/6, Loss: 0.8649450540542603
Iteration: 4/6, Loss: 0.8568985462188721
Iteration: 5/6, Loss: 0.7876611351966858
Iteration: 6/6, Loss: 0.659570038318634
Epoch: 2/10
Iteration: 1/6, Loss: 0.5371824502944946
Iteration: 2/6, Loss: 0.3628958761692047
Iteration: 3/6, Loss: 0.2279462367296219
Iteration: 4/6, Loss: 0.19122938811779022
Iteration: 5/6, Loss: 0.11094227433204651
Iteration: 6/6, Loss: 0.43904152512550354
Epoch: 3/10
Iteration: 1/6, Loss: 0.2262747883796692
Iteration: 2/6, Loss: 0.04857257381081581
Iteration: 3/6, Loss: 0.1275649219751358
Iteration: 4/6, Loss: 1.5481326580047607
Iteration: 5/6, Loss: 0.14442549645900726
Iteratio